In [17]:
%matplotlib inline
import pandas as pd
import re
import seaborn as sns
import subprocess
import bitmath

ssh_server = 'skchoudh@hpc-cmb.usc.edu'
connection_status=subprocess.check_output('ssh -t {} "qstat cmb -n -1"'.format(ssh_server),shell=True).strip()

with open('test/connection_status.log','w') as fh:
    fh.write(connection_status)
    
with open('test/connection_status.log') as fh:
    content = fh.readlines()  
    content_lines = list(map(lambda x: re.sub( '\s+', '\t', x ), content))

with open('test/connection_status.log.clean', 'w') as fh:
    fh.write('\n'.join(content_lines[5:]))

df = pd.read_table('test/connection_status.log.clean', header=None)
df.drop(df.columns[[12]], axis=1, inplace=True)
df.columns=['job_id', 'user', 'queue', 'jobname', 'session_id',
            'nds', 'tsk', 'req_mem', 'req_time', 'status', 'elap_time', 'nodes']
df.req_mem = df.req_mem.map(lambda x: bitmath.parse_string(x.upper()).to_GB())
df_grouped = df.groupby(['status','user'])
#df_grouped.agg(['mean', 'count'])
df_info = df_grouped.aggregate({'nds': 'sum', 'req_mem': 'sum', 'tsk': 'sum'})

df_info.sort_values(by=['req_mem', 'nds', 'tsk'], ascending=False).groupby('status')

KeyError: 'status'

In [19]:
df_info = df_grouped[['req_mem', 'nds', 'tsk']].transform(sum)

In [22]:
df_sorted = df_info.sort_values(by=['req_mem', 'nds', 'tsk'], ascending=False)

ValueError: Cannot index with multidimensional key

In [23]:
df_index = df.ix[df_sorted.index]

In [26]:
df_sorted_groups = df_index.groupby(['status', 'user'])

In [28]:
df_sorted_groups.groups()

TypeError: 'dict' object is not callable